In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
from malaya_speech.torch_model.vits_v2.model_infer import SynthesizerTrn
from malaya_boilerplate.train.config import HParams
import json

2023-08-26 08:33:04.423064: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-26 08:33:04.495888: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-26 08:33:05.000671: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-08-26 08:33:05.000704: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

/home/husein/.local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


`pyaudio` is not available, `malaya_speech.streaming.pyaudio` is not able to use.


In [3]:
config = 'malay_vits/ms-husein.json'

In [4]:
with open(config) as fopen:
    hps = HParams(**json.load(fopen))

In [5]:
from malaya_speech.utils.text import TTS_SYMBOLS
import torch

In [6]:
model = SynthesizerTrn(len(TTS_SYMBOLS),
                                   hps.data.n_mel_channels,
                                   hps.train.segment_size // hps.data.hop_length,
                                   **hps.model)

In [24]:
!ls malay_vits/logs/husein-v2/G_*.pth

malay_vits/logs/husein-v2/G_100000.pth	malay_vits/logs/husein-v2/G_80000.pth
malay_vits/logs/husein-v2/G_105000.pth	malay_vits/logs/husein-v2/G_85000.pth
malay_vits/logs/husein-v2/G_110000.pth	malay_vits/logs/husein-v2/G_90000.pth
malay_vits/logs/husein-v2/G_115000.pth	malay_vits/logs/husein-v2/G_95000.pth
malay_vits/logs/husein-v2/G_75000.pth


In [25]:
pth = 'malay_vits/logs/husein-v2/G_115000.pth'

In [26]:
model.load_state_dict(torch.load(pth, map_location=torch.device('cpu'))['model'])

<All keys matched successfully>

In [27]:
_ = model.eval()

In [28]:
from malaya_speech.torch_model.vits.commons import intersperse
import malaya_speech

In [12]:
normalizer = malaya_speech.utils.text.TextIDS(pad_to = None, is_lower = False)

In [35]:
text = 'Haqkiem adalah pelajar tahun akhir yang mengambil Ijazah Sarjana Muda Sains Komputer Kecerdasan Buatan.'

In [14]:
text2 = 'KUALA TERENGGANU - Kerajaan didakwa ketandusan isu sehingga mengaitkan pemimpin Perikatan Nasional (PN) dengan kenyataan berunsur perkauman.'

In [36]:
t, ids = normalizer.normalize(text.lower())
t

'haqkiem adalah pelajar tahun akhir yang mengambil ijazah sarjana muda sains komputer kecerdasan buatan .'

In [37]:
ids = intersperse(ids, 0)

In [38]:
ids = torch.LongTensor(ids)
ids_lengths = torch.LongTensor([ids.size(0)])
ids = ids.unsqueeze(0)

In [39]:
audio = model.infer(
    ids,
    ids_lengths,
    noise_scale=0.,
    noise_scale_w=0.,
    length_scale=1.0,
)

In [40]:
audio = audio[0][0,0].detach().numpy()

In [41]:
import IPython.display as ipd
ipd.Audio(audio, rate = 22050)

In [42]:
torch.save(model.state_dict(), 'husein.pth')

In [43]:
from malaya_boilerplate.huggingface import upload_dict

In [ ]:
files_mapping = {'husein.pth': 'model.pth',
                config: 'config.json'}
upload_dict(model = 'VITS-V2-husein', files_mapping = files_mapping, username = 'mesolitica')

409 Client Error: Conflict for url: https://huggingface.co/api/repos/create (Request ID: Root=1-64e9913f-5513e6cb1bb1a01f4a0d884e;22589d6c-4c93-4c3b-9539-e91ec91d6e21)

You already created this model repo


husein.pth:   0%|          | 0.00/145M [00:00<?, ?B/s]